# pytest.raises

In [14]:
%%ipytest
import pytest

def parse_pos_int(s):
    n = int(s)
    if n < 0:
        raise ValueError("No negativity allowed, positive vibes only!")
    return n

def test_ok():
    assert parse_pos_int("5") == 5

def test_not_an_int():
    with pytest.raises(ValueError):
        parse_pos_int("x")

def test_negative():
    with pytest.raises(ValueError, match="No negativity allowed"):
        parse_pos_int("-5")

======================================= test session starts ========================================
collected 3 items

t_6947442cdcf74a3696aef7e735f8c8a1.py ..F                                                    [100%]

============================================= FAILURES =============================================
__________________________________________ test_negative ___________________________________________

    def test_negative():
        with pytest.raises(ValueError, match="No negativity allowed"):
>           parse_pos_int("-5a")

/tmp/ipykernel_486516/4055813075.py:18: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

s = '-5a'

    def parse_pos_int(s):
>       n = int(s)
E       ValueError: invalid literal for int() with base 10: '-5a'

/tmp/ipykernel_486516/4055813075.py:4: ValueError

During handling of the above exception, another exception occurred:

    def test_negative():
>       with pytest.raises(ValueErr

# Marks

Audience question: How to parametrize but only have one test case raise something?

In [20]:
%%ipytest
import pytest

@pytest.mark.parametrize("s, n", [
    ("5", 5),
    ("8", 8),
])
def test_good(s, n):
    assert parse_pos_int(s) == n


@pytest.mark.parametrize("s, message", [
    ("-1", "No negativity allowed"),
    ("a", "invalid literal for int"),
])
def test_bad(s, message):
    with pytest.raises(ValueError, match=message):
        parse_pos_int(s)

======================================= test session starts ========================================
collected 4 items

t_6947442cdcf74a3696aef7e735f8c8a1.py ....                                                   [100%]

======================================== 4 passed in 0.02s =========================================


In [24]:
%%ipytest
import pytest
from contextlib import nullcontext

@pytest.mark.parametrize("inp, expectation", [
    ("5", nullcontext()),
    ("8", nullcontext()),
    ("-1", pytest.raises(ValueError)),
    ("a", pytest.raises(ValueError)),
])
def test_smoke(inp, expectation):
    with expectation:
        parse_pos_int(inp)

======================================= test session starts ========================================
collected 4 items

t_6947442cdcf74a3696aef7e735f8c8a1.py ....                                                   [100%]

======================================== 4 passed in 0.02s =========================================


[Flaky tests · Issue #5390 · qutebrowser/qutebrowser](https://github.com/qutebrowser/qutebrowser/issues/5390)

# Fixtures

In [32]:
%%ipytest

import pytest

from rpncalc.utils import Config

def get_prompt_from_marker(prompt):
    return prompt


@pytest.fixture
def config(request: pytest.FixtureRequest) -> Config:
    marker = request.node.get_closest_marker("prompt")
    if marker is None:
        return Config(prompt=">")
    return Config(prompt=get_prompt_from_marker(*marker.args, **marker.kwargs))


def test_normal(config: Config):
    assert config.prompt == ">"



@pytest.mark.prompt("rpn>", blabla=42)
def test_marker(config: Config):
    assert config.prompt == "rpn>"


### possible alternative approach for arg validation

def prompt_mark(prompt: str):
    return pytest.mark.prompt(prompt)



======================================= test session starts ========================================
collected 2 items

t_6947442cdcf74a3696aef7e735f8c8a1.py .E                                                     [100%]

============================================== ERRORS ==============================================
__________________________________ ERROR at setup of test_marker ___________________________________

request = <SubRequest 'config' for <Function test_marker>>

    @pytest.fixture
    def config(request: pytest.FixtureRequest) -> Config:
        marker = request.node.get_closest_marker("prompt")
        if marker is None:
            return Config(prompt=">")
>       return Config(prompt=get_prompt_from_marker(*marker.args, **marker.kwargs))
E       TypeError: get_prompt_from_marker() got an unexpected keyword argument 'blabla'

/tmp/ipykernel_486516/1425000994.py:14: TypeError
===================================== short test summary info =============================

# Links

- [GitHub - boxed/mutmut: Mutation testing system](https://github.com/boxed/mutmut)
- [flakytest.dev](https://flakytest.dev/)